# Binary Class Credit score Prediction.

person_age -	Age

person_income -	Annual Income

person_home_ownershipHome ownership

person_emp_length 	Employment length (in years)

loan_intent 	Loan intent

loan_grade 	Loan grade

loan_amnt 	Loan amount

loan_int_rate 	Interest rate

loan_status 	Loan status (0 is non default 1 is default)

loan_percent_income 	Percent income

cb_person_default_on_file 	Historical default

cb_preson_cred_hist_length 	Credit history length

**Import Libraries  and the Data**

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import colorama
from colorama import Fore, Back, Style

#sklearn package
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

##keras package
from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from keras.utils import to_categorical

##lime package
import lime
from lime import lime_tabular

import warnings
warnings.filterwarnings("ignore")
import pickle

In [ ]:
#import the data
df = pd.read_csv("credit_risk_dataset.csv")
df.head()

In [ ]:
#what is does our data composed of
df.info()

In [ ]:
#the different categpries in each of the categorical variables 

print(Style.BRIGHT+ 'Summary of Credit Risk Data:' + Style.RESET_ALL)
print(Style.BRIGHT+ '--------------------------' + Style.RESET_ALL)
print(Fore.BLACK+ Style.BRIGHT+ 'person_home_ownership' + Style.RESET_ALL+'           :',\
      Fore.RED+ Style.BRIGHT+ str(4) + Style.RESET_ALL + ' (Rent, Own, Mortgage, Other )')
print(Fore.BLACK+ Style.BRIGHT+ 'Loan_intent' + Style.RESET_ALL+'          :',\
      Fore.RED+ Style.BRIGHT+ str(6) + Style.RESET_ALL + ' (Personal, Education, Medical,Venture, Home Improvemnet, Debt Consolidation)')
print(Fore.BLACK+ Style.BRIGHT+ 'loan_grade' + Style.RESET_ALL+'       :',\
      Fore.RED+ Style.BRIGHT+ str(7) + Style.RESET_ALL + ' (A, B, C, D, E, F, G)')
print(Fore.BLACK+ Style.BRIGHT+ 'cb_person_default_on_file' + Style.RESET_ALL+'        :',\
      Fore.RED+ Style.BRIGHT+ str(2) + Style.RESET_ALL + ' (Y, N)')
print(Fore.BLACK+ Style.BRIGHT+ 'Loan_Status' + Style.RESET_ALL+'      :',\
      Fore.RED+ Style.BRIGHT+ str(2) + Style.RESET_ALL + ' (1, 0)')

In [ ]:
duplicated = df.duplicated()
df[duplicated]

In [ ]:
df.query("person_age==23 & person_income==42000 &\
person_home_ownership=='RENT'& loan_int_rate==9.99")

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df_new = df

In [ ]:
#use correlation to check collinearity
cor = df_new.corr(method='spearman')
plt.figure(figsize=(12,5))
sns.heatmap(cor, cmap=plt.cm.winter_r,annot=True)
plt.show() 

In [ ]:
sns.countplot(x ='person_home_ownership', hue='loan_status',data=df)
plt.show()

In [ ]:
sns.countplot(data=df, y="loan_intent", hue="loan_status")
plt.show()

In [ ]:
sns.countplot(x ='cb_person_default_on_file', hue='loan_status',data=df)
plt.show()

In [ ]:
sns.countplot(x ='loan_grade', hue='loan_status',data=df)
plt.show()

In [ ]:
#drop the variables with a high correlation of more than 0.8
df_new = df_new.drop(['person_age'], axis=1)

In [ ]:
#encode the categorical variables
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder()

for col in df_new.columns:
    if df_new[col].dtype == 'object':
        df_new[col] = le.fit_transform(df_new[col])

In [ ]:
#define an estimator
lr = LinearRegression()
imp = IterativeImputer(estimator=lr, verbose=2, max_iter=30, tol=1e-10, imputation_order='roman')

#imputer = IterativeImputer(random_state=100, max_iter=10)
K = imp.fit_transform(df_new)

In [ ]:
K1 = pd.DataFrame(K)

In [ ]:
col_names = {i:df_new.columns[i] for i in range(len(list(df_new.columns)))}

In [ ]:
K1.rename(columns=col_names, inplace=True)
df_new = K1

In [ ]:
df_new.head()

In [ ]:
df_new.isna().sum()

In [ ]:
#divide the data into input matrix and output separately
X = df_new.drop(['loan_status'], axis=1)
y= df_new['loan_status']

In [ ]:
#check for similarity in these three variables
X[['person_income', 'loan_amnt', 'loan_percent_income']].head()

In [ ]:
X.drop('loan_percent_income', axis=1,inplace=True)

In [ ]:
X.shape

In [ ]:
#use RFECV for feature selection

# Create a decision tree classifier
estimator = DecisionTreeClassifier()
  
# find the optimal number of features
selector = RFECV(estimator, cv=5)
selector = selector.fit(X, y)
  
# Print the optimal number of features
print("Optimal number of features: %d" % selector.n_features_)
  
# Print the selected features
print("Selected features: %s" % selector.support_)

In [ ]:
select_rfecv = X.columns[selector.support_].tolist()
select_rfecv

In [ ]:
 #drop the irrelevant features not chosen by the classifier
X_new = X.drop(['cb_person_default_on_file'], axis=1)

In [ ]:
# define standard scaler
scaler = StandardScaler()
  
# transform data
X_scaled = scaler.fit_transform(X_new)

**Splitting the data**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state=0)
len(x_train), len(y_train), len(x_test), len(y_test)

In [ ]:
y_train.value_counts().plot.pie(autopct='%1.1f%%', textprops ={'fontsize':10})
plt.show()

In [ ]:
#we need to balance the training set
#counter is used to count the values
counter = Counter(y_train)
print('Before', counter)

#define smote
smt = SMOTE()
X_train_smt, y_train_smt = smt.fit_resample(x_train, y_train)
counter = Counter(y_train_smt)
print('After', counter)

**Model Building**

In [ ]:
#build a logistic regression model
#before balancing the data

model = LogisticRegression()
model.fit(x_train, y_train)

In [ ]:
#after balancing the data
logreg_smt = LogisticRegression()
logreg_smt.fit(X_train_smt, y_train_smt)

In [ ]:
#models prediction
pred_y = model.predict(x_test)
y_pred_smt = logreg_smt.predict(x_test)

In [ ]:
#Get the confusion matrix
cf_matrix = confusion_matrix(y_test, pred_y)

#print(cf_matrix)
ax = sns.heatmap(cf_matrix, annot=True,fmt='d', cmap='Blues')

# set x-axis label and ticks. 
ax.set_xlabel("Predicted Value", fontsize=14, labelpad=20)
ax.xaxis.set_ticklabels(['non defaulter', 'defaulter'])

# set y-axis label and ticks
ax.set_ylabel("Actual Value", fontsize=14, labelpad=20)
ax.yaxis.set_ticklabels(['non defaulter', 'defaulter'])

# set plot title
ax.set_title("Confusion Matrix Before Balancing", fontsize=14, pad=20)
plt.show()

In [ ]:
#the classification report before balancing the data
report = classification_report(y_test, pred_y)
print(report)

In [ ]:
#plotting confusionmatrix after balancing the data
#Get the confusion matrix
cf_matrixb = confusion_matrix(y_test, y_pred_smt)
#print(cf_matrix)
ax1 = sns.heatmap(cf_matrixb, annot=True,fmt='d', cmap='Blues')
# set x-axis label and ticks. 
ax1.set_xlabel("Predicted Value", fontsize=14, labelpad=20)
ax1.xaxis.set_ticklabels(['non defaulter', 'defaulter'])

# set y-axis label and ticks
ax1.set_ylabel("Actual Value", fontsize=14, labelpad=20)
ax1.yaxis.set_ticklabels(['non defaulter', 'defaulter'])

# set plot title
ax1.set_title("Confusion Matrix After Balancing", fontsize=14, pad=20)
plt.show()

In [ ]:
#the classification report of after data balancing
report = classification_report(y_test, y_pred_smt)
print(report)

In [ ]:
#we need other metrices which are not in the classification report

print(f"Balanced Accuracy of the unbalanced data is: {balanced_accuracy_score(y_test, pred_y)}")
print(f"Balanced Accuracy of the balanced data is: {balanced_accuracy_score(y_test, y_pred_smt)}")

print(f"AUC-ROC Score of the unbalanced data is: {roc_auc_score(y_test, pred_y)}")
print(f"AUC-ROC Score of the balanced data is: {roc_auc_score(y_test, y_pred_smt)}")

print(f"Matthews Coefficient of the unbalanced data is: {matthews_corrcoef(y_test, pred_y)}")
print(f"Matthews Coefficient of the balanced data is: {matthews_corrcoef(y_test, y_pred_smt)}")

In [ ]:
# predict probabilities
pred_prob1 = model.predict_proba(x_test)
pred_prob2 = logreg_smt.predict_proba(x_test)

In [ ]:
# auc scores
auc_score1 = roc_auc_score(y_test, pred_prob1[:,1])
auc_score2 = roc_auc_score(y_test, pred_prob2[:,1])

print(auc_score1, auc_score2)

**Multilayer Perceptron**

In [ ]:
#we need to convert the values to categorical
y_train1 = to_categorical(y_train)
y_train_smt1 = to_categorical(y_train_smt)
y_test1 = to_categorical(y_test)

In [ ]:
model_1 = Sequential()
model_1.add(Dense(units=8, activation='relu', input_dim=x_train.shape[1]))
model_1.add(layers.Dropout(0.1))
model_1.add(Dense(units=6, activation='relu' ))
model_1.add(layers.Dropout(0.1))
model_1.add(Dense(units=2, activation='sigmoid'))
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_1.summary()

In [ ]:
#train the model
history = model_1.fit(x_train, y_train1, validation_split=0.2, epochs=50, batch_size=132)

In [ ]:
#model after balancing the data
model_2 = Sequential()
model_2.add(Dense(units=8, activation='relu', input_dim=x_train.shape[1]))
model_2.add(layers.Dropout(0.1))
model_2.add(Dense(units=6, activation='relu' ))
model_2.add(layers.Dropout(0.1))
model_2.add(Dense(units=2, activation='sigmoid'))
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_2.summary()

In [ ]:
history1 = model_2.fit(X_train_smt, y_train_smt1,validation_split=0.2, epochs=50, batch_size=32)

In [ ]:
# Evaluate model
loss, accuracy = model_1.evaluate(x_test, y_test1)
 
print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
# Evaluate model
loss, accuracy2 = model_2.evaluate(x_test, y_test1)
 
print('Test loss:', loss)
print('Test accuracy:', accuracy2)

In [ ]:
# Make prediction on test dataset
pred_y = model_1.predict(x_test)
pred_y2 = model_2.predict(x_test)

In [ ]:
pred_y1= np.argmax(pred_y, axis=1)


In [ ]:
y_true = np.argmax(y_test1, axis=1)
y_true

In [ ]:
pred_y2= np.argmax(pred_y2, axis=1)

In [ ]:
report = classification_report(y_true, pred_y1)
print(report)

In [ ]:
#Get the confusion matrix
cf_matrix = confusion_matrix(y_true, pred_y1)
#print(cf_matrix)
ax = sns.heatmap(cf_matrix, annot=True,fmt='d', cmap='Blues')
# set x-axis label and ticks. 
ax.set_xlabel("Predicted Value", fontsize=14, labelpad=20)
ax.xaxis.set_ticklabels(['non defaulter', 'defaulter'])

# set y-axis label and ticks
ax.set_ylabel("Actual Value", fontsize=14, labelpad=20)
ax.yaxis.set_ticklabels(['non defaulter', 'defaulter'])

# set plot title
ax.set_title("Confusion Matrix Before Balancing", fontsize=14, pad=20)
#plt.savefig('ConMat_nn1.png')
plt.show()

In [ ]:
report = classification_report(y_true, pred_y2)
print(report)

In [ ]:
#Get the confusion matrix
cf_matrix = confusion_matrix(y_true, pred_y2)
#print(cf_matrix)
ax = sns.heatmap(cf_matrix, annot=True,fmt='d', cmap='Blues')
# set x-axis label and ticks. 
ax.set_xlabel("Predicted Value", fontsize=14, labelpad=20)
ax.xaxis.set_ticklabels(['non defaulter', 'defaulter'])

# set y-axis label and ticks
ax.set_ylabel("Actual Value", fontsize=14, labelpad=20)
ax.yaxis.set_ticklabels(['non defaulter', 'defaulter'])

# set plot title
ax.set_title("Confusion Matrix After Balancing", fontsize=14, pad=20)
#plt.savefig('ConMat_nn2.png')
plt.show()

In [ ]:
print(f"Accuracy of the unbalanced data is: {accuracy_score(y_true, pred_y1)}")
print(f"Accuracy of the balanced data is: {accuracy_score(y_true, pred_y2)}")

print(f"Balanced Accuracy of the unbalanced data is: {balanced_accuracy_score(y_true, pred_y1)}")
print(f"Balanced Accuracy of the balanced data is: {balanced_accuracy_score(y_true, pred_y2)}")

print(f"Matthews Coefficient of the unbalanced data is: {matthews_corrcoef(y_true, pred_y1)}")
print(f"Matthews Coefficient of the balanced data is: {matthews_corrcoef(y_true, pred_y2)}")

In [ ]:
roc1 = roc_auc_score(y_true, pred_y1)
roc2 = roc_auc_score(y_true, pred_y2)
roc1, roc2

In [ ]:
feature_names = X_new.columns
# Obtain the feature importances
importances = np.sum(np.abs(model_1.get_weights()[0]), axis=1)

# Sort the feature importances in descending order and get the corresponding feature names
sorted_indices = np.argsort(importances)[::-1]
sorted_importances = importances[sorted_indices]
sorted_feature_names = [feature_names[i] for i in sorted_indices]

# Create a horizontal bar plot of the feature importances
fig, ax = plt.subplots()
ax.barh(range(len(sorted_importances)), sorted_importances, tick_label=sorted_feature_names)
ax.set_xlabel('Feature Importance')
ax.set_ylabel('Feature')
plt.title('Feature on Imbalanced data')
plt.savefig('feature imp b_n1.png')
plt.show()

In [ ]:
feature_names = X_new.columns
# Obtain the feature importances
importances = np.sum(np.abs(model_2.get_weights()[0]), axis=1)

# Sort the feature importances in descending order and get the corresponding feature names
sorted_indices = np.argsort(importances)[::-1]
sorted_importances = importances[sorted_indices]
sorted_feature_names = [feature_names[i] for i in sorted_indices]

# Create a horizontal bar plot of the feature importances
fig, ax = plt.subplots()
ax.barh(range(len(sorted_importances)), sorted_importances, tick_label=sorted_feature_names)
ax.set_xlabel('Feature Importance')
ax.set_ylabel('Feature')
plt.title('Feature on MLP')
plt.savefig('feature imp b_n2.png')
plt.show()

In [ ]:
# Save the trained model as a pickle string.
#saved_model_n1 = pickle.dumps(model_1)
#saved_model_n2 = pickle.dumps(model_2)

# Load the pickled model
#load_model1 = pickle.loads(saved_model)
  
# Use the loaded pickled model to make predictions
#load_model1.predict(X_test)

**LIME**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.2, random_state=0)
len(x_train), len(y_train), len(x_test), len(y_test)

In [ ]:
counter = Counter(y_train)
print('Before', counter)

smt = SMOTE()
X_train_smt, y_train_smt = smt.fit_resample(x_train, y_train)
counter = Counter(y_train_smt)
print('After', counter)

In [ ]:
#imbalanced data
explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(x_train),
    feature_names=x_train.columns,
    class_names=['non defaulter', 'defaulter'],
    mode='classification'
)

In [ ]:
exp = explainer.explain_instance(
    data_row=x_test.iloc[4], 
    predict_fn=model_1.predict
    
)

exp.show_in_notebook(show_table=True)

In [ ]:
#balanced data
explainer1 = lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train_smt),
    feature_names=X_train_smt.columns,
    class_names=['non defaulter', 'defaulter'],
    mode='classification'
)

In [ ]:
exp = explainer1.explain_instance(
    data_row=x_test.iloc[4], 
    predict_fn=model_2.predict
)

exp.show_in_notebook(show_table=True)